### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [47]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [48]:
# There are no empty cells in the data
school_data.count()
student_data.count()

total_number_of_schools  = len(school_data)
total_number_of_students = len(student_data)
total_budget = school_data['budget'].sum()
average_math_score = student_data['math_score'].mean()
average_reading_score = student_data['reading_score'].mean()
overall_passing_rate  = (average_math_score + average_reading_score) / 2
passing_math_students = ['approved' for grade in student_data['math_score'] if grade >= 70]
passing_reading_students = ['approved' for grade in student_data['reading_score'] if grade >= 70]
math_approved_percentage = len(passing_math_students) / total_number_of_students
reading_approved_percentage = len(passing_reading_students) / total_number_of_students

data = {
    '#  of schools'         : [total_number_of_schools],
    '# of students'         : total_number_of_students,
    'Total  budget'         : total_budget,
    'Average   math  score' : average_math_score,
    'Average reading score' : average_reading_score,
    'Overall passing  rate' : overall_passing_rate,
    'Approved % math'       : math_approved_percentage    * 100,
    'Approved % reading'    : reading_approved_percentage * 100
}

summary_df = pd.DataFrame(data)
summary_df

,# of schools,# of students,Total budget,Average math score,Average reading score,Overall passing rate,Approved % math,Approved % reading
0,15,39170,24649428,78.985371,81.87784,80.431606,74.980853,85.805463


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [49]:
# Group school data by school
aggregation_functions = {'type':'first','budget':'first'}
school_data_by_school_name = school_data.groupby(school_data['school_name']).aggregate(aggregation_functions)


# Count the number of students that approved for each school
def studentApproved(score):
    return (score>=70).sum()
approved_students_by_school = student_data.groupby(student_data['school_name']).aggregate({'math_score'   : studentApproved
                                                                                          ,'reading_score': studentApproved })

# Group student data by school
aggregation_functions = { 'student_name'  :'count'
                         ,'math_score'   : 'mean'
                         ,'reading_score': 'mean'
                        }
student_data_by_school_name = student_data.groupby(student_data['school_name']).aggregate(aggregation_functions)


# Merge both tables
top_performing_schools = pd.concat([student_data_by_school_name,school_data_by_school_name],axis=1)
top_performing_schools = top_performing_schools.rename(columns={'student_name'   : 'Total students'
                                                        ,'math_score'    : 'Average Math Score'
                                                        ,'reading_score' : 'Average Reading Score'
                                                        ,'type'  : 'School Type'
                                                        ,'budget': 'School Budget'})
# Calculate missing columns
top_performing_schools['Per Student Budget']     = (top_performing_schools['School Budget']      / top_performing_schools['Total students'])
top_performing_schools['% Passing Math']         = (approved_students_by_school['math_score']    / top_performing_schools['Total students'])    * 100
top_performing_schools['% Passing Reading']      = (approved_students_by_school['reading_score'] / top_performing_schools['Total students']) * 100
top_performing_schools['% Overall Passing Rate'] = (top_performing_schools['% Passing Math']     + top_performing_schools['% Passing Reading']) / 2 

# Sort by top overall passing rate
top_performing_schools = top_performing_schools.sort_values(by='% Overall Passing Rate',ascending=False)
top_performing_schools.head(5)

,Total students,Average Math Score,Average Reading Score,School Type,School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,1858,83.061895,83.975780,Charter,1081356,582.0,94.133477,97.039828,95.586652
Thomas High School,1635,83.418349,83.848930,Charter,1043130,638.0,93.272171,97.308869,95.290520
Pena High School,962,83.839917,84.044699,Charter,585858,609.0,94.594595,95.945946,95.270270
Griffin High School,1468,83.351499,83.816757,Charter,917500,625.0,93.392371,97.138965,95.265668
Wilson High School,2283,83.274201,83.989488,Charter,1319574,578.0,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [50]:
top_performing_schools = top_performing_schools.sort_values(by='% Overall Passing Rate',ascending=True)
top_performing_schools.head(5)

,Total students,Average Math Score,Average Reading Score,School Type,School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,3999,76.842711,80.744686,District,2547363,637.0,66.366592,80.220055,73.293323
Figueroa High School,2949,76.711767,81.158020,District,1884411,639.0,65.988471,80.739234,73.363852
Huang High School,2917,76.629414,81.182722,District,1910635,655.0,65.683922,81.316421,73.500171
Johnson High School,4761,77.072464,80.966394,District,3094650,650.0,66.057551,81.222432,73.639992
Ford High School,2739,77.102592,80.746258,District,1763916,644.0,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [51]:


# approved_students_by_school_9th  = student_data.groupby(student_data['school_name']).aggregate({'grade' : studentGrade_9th}).rename ( columns = {'grade': '9th'})
score = 'math_score'
grade_9th_dt  = student_data[student_data['grade'] == '9th']
grade_10th_dt = student_data[student_data['grade'] == '10th']
grade_11th_dt = student_data[student_data['grade'] == '11th']
grade_12th_dt = student_data[student_data['grade'] == '12th']
reading_averages_by_school_9th   = student_data.groupby(grade_9th_dt ['school_name']).aggregate({score : 'mean'}).rename ( columns = {score: '9th'})
reading_averages_by_school_10th  = student_data.groupby(grade_10th_dt['school_name']).aggregate({score : 'mean'}).rename ( columns = {score: '10th'})
reading_averages_by_school_11th  = student_data.groupby(grade_11th_dt['school_name']).aggregate({score : 'mean'}).rename ( columns = {score: '11th'})
reading_averages_by_school_12th  = student_data.groupby(grade_12th_dt['school_name']).aggregate({score : 'mean'}).rename ( columns = {score: '12th'})

reading_averages_by_school = pd.concat([reading_averages_by_school_9th
                                       ,reading_averages_by_school_10th
                                       ,reading_averages_by_school_11th
                                       ,reading_averages_by_school_12th],axis=1)
reading_averages_by_school.head(10)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [52]:
# approved_students_by_school_9th  = student_data.groupby(student_data['school_name']).aggregate({'grade' : studentGrade_9th}).rename ( columns = {'grade': '9th'})
score = 'reading_score'
grade_9th_dt  = student_data[student_data['grade'] == '9th']
grade_10th_dt = student_data[student_data['grade'] == '10th']
grade_11th_dt = student_data[student_data['grade'] == '11th']
grade_12th_dt = student_data[student_data['grade'] == '12th']
reading_averages_by_school_9th   = student_data.groupby(grade_9th_dt ['school_name']).aggregate({score : 'mean'}).rename ( columns = {score: '9th'})
reading_averages_by_school_10th  = student_data.groupby(grade_10th_dt['school_name']).aggregate({score : 'mean'}).rename ( columns = {score: '10th'})
reading_averages_by_school_11th  = student_data.groupby(grade_11th_dt['school_name']).aggregate({score : 'mean'}).rename ( columns = {score: '11th'})
reading_averages_by_school_12th  = student_data.groupby(grade_12th_dt['school_name']).aggregate({score : 'mean'}).rename ( columns = {score: '12th'})

reading_averages_by_school = pd.concat([reading_averages_by_school_9th
                                       ,reading_averages_by_school_10th
                                       ,reading_averages_by_school_11th
                                       ,reading_averages_by_school_12th],axis=1)
reading_averages_by_school.head(10)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [53]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

top_performing_schools["Student School Spending"] = pd.cut(top_performing_schools['Per Student Budget'], spending_bins, labels=group_names)
# df1['value2'] = df1['id'].map(df2.set_index('id')['value2'])


student_data['Student School Spending'] = student_data['school_name'].map(top_performing_schools['Student School Spending'])

approved_students_by_school_spending = student_data.groupby(student_data['Student School Spending']).aggregate({'math_score'   : studentApproved
                                                                                                                ,'reading_score': studentApproved })
aggregation_functions = {
                          'math_score'   : 'mean'
                         ,'reading_score': 'mean'
                        }
scores_by_school_spending = student_data.groupby(student_data['Student School Spending']).aggregate(aggregation_functions)
total_students_by_school_spending = student_data.groupby(student_data['Student School Spending']).aggregate({'student_name'  : 'count'})
scores_by_school_spending['% Passing Math']    = (approved_students_by_school_spending['math_score']    /  total_students_by_school_spending['student_name']) * 100
scores_by_school_spending['% Passing Reading'] = (approved_students_by_school_spending['reading_score'] /  total_students_by_school_spending['student_name']) * 100
scores_by_school_spending['Overall Passing Rate'] = (scores_by_school_spending['% Passing Math']  +scores_by_school_spending['% Passing Reading'])  / 2
scores_by_school_spending = scores_by_school_spending.rename(columns={'math_score' : 'Average Math Score'
                                                                     , 'reading_score' : 'Average Reading Score'})
scores_by_school_spending


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Student School Spending,,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


In [54]:

school_data


,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


## Scores by School Size

* Perform the same operations as above, based on school size.

In [55]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_data["School Size type"] = pd.cut(school_data['size'], size_bins, labels=group_names)
student_data['Student School Size Type'] = student_data['school_name'].map(school_data.set_index('school_name')['School Size type'])
student_data
approved_students_by_school_size = student_data.groupby(student_data['Student School Size Type']).aggregate({'math_score'   : studentApproved
                                                                                                                ,'reading_score': studentApproved })

aggregation_functions = {
                          'math_score'   : 'mean'
                         ,'reading_score': 'mean'
                        }
scores_by_school_size = student_data.groupby(student_data['Student School Size Type']).aggregate(aggregation_functions)

total_students_by_school_size = student_data.groupby(student_data['Student School Size Type']).aggregate({'student_name'  : 'count'})
scores_by_school_size['% Passing Math']    = (approved_students_by_school_size['math_score']    /  total_students_by_school_size['student_name']) * 100
scores_by_school_size['% Passing Reading'] = (approved_students_by_school_size['reading_score'] /  total_students_by_school_size['student_name']) * 100
scores_by_school_size['Overall Passing Rate'] = (scores_by_school_size['% Passing Math']  + scores_by_school_size['% Passing Reading'])  / 2
scores_by_school_size = scores_by_school_size.rename(columns={'math_score' : 'Average Math Score'
                                                                     , 'reading_score' : 'Average Reading Score'})

scores_by_school_size


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Student School Size Type,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


## Scores by School Type

* Perform the same operations as above, based on school type.